In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

DATA_PATH = Path("CAISOHourlyLoadCSV.csv")
CUTOFF_END = pd.Timestamp("2026-02-19 23:59:00", tz = "America/Los_Angeles")

C:\Users\ayerh\OneDrive\Documents\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Task 1
## Part a)

In [5]:
df = pd.read_csv(DATA_PATH)

d = pd.to_datetime(df["Date"], errors = "coerce")
he = pd.to_numeric(df["Hour"], errors = "coerce")
load = pd.to_numeric(df["CAISO Load (MW)"], errors = "coerce")
tmp = pd.DataFrame({"date": d, "he": he, "load": load}).dropna()

base = tmp["date"].dt.tz_localize("America/Los_Angeles")
ts = base + pd.to_timedelta(tmp["he"].astype(int), unit = "h")

hourly = pd.Series(tmp["load"].values, index = ts).sort_index()
hourly = hourly[~hourly.index.duplicated(keep = "last")].asfreq("H")

hourly = hourly.loc[hourly.index <= CUTOFF_END]

monthly_peaks = hourly.resample("MS").max().dropna()

target_ms = pd.Timestamp("2026-03-01", tz = "America/Los_Angeles")
last_ms = monthly_peaks.index.max()
steps = int(target_ms.to_period("M").ordinal - last_ms.to_period("M").ordinal)
if steps <= 0:
    raise ValueError("Your data includes March 2026 or later (violates cutoff).")

model = ETSModel(monthly_peaks, error = "add", trend = "add", seasonal = None)
fit = model.fit(disp = False)

L_star = float(fit.forecast(steps = steps).iloc[-1])
print("Months of history used:", len(monthly_peaks))
print(f"L* (March 2026 peak MW point forecast): {L_star:,.0f} MW")

Months of history used: 13
L* (March 2026 peak MW point forecast): -12,369 MW


C:\Users\ayerh\AppData\Local\Temp\ipykernel_22888\1834639145.py:24: UserWarning: Converting to Period representation will drop timezone information.
  steps = int(target_ms.to_period("M").ordinal - last_ms.to_period("M").ordinal)


## Part b)

In [6]:
df = pd.read_csv(DATA_PATH)

d = pd.to_datetime(df["Date"], errors = "coerce")
he = pd.to_numeric(df["Hour"], errors = "coerce")
load = pd.to_numeric(df["CAISO Load (MW)"], errors = "coerce")
tmp = pd.DataFrame({"date": d, "he": he, "load": load}).dropna()

base = tmp["date"].dt.tz_localize("America/Los_Angeles")
ts = base + pd.to_timedelta(tmp["he"].astype(int), unit = "h")

hourly = pd.Series(tmp["load"].values, index = ts).sort_index()
hourly = hourly[~hourly.index.duplicated(keep = "last")].asfreq("H")
hourly = hourly.loc[hourly.index <= CUTOFF_END].dropna()

march = hourly[hourly.index.month == 3]
if march.empty:
    raise RuntimeError("No March history found in your data; cannot forecast d* with this heuristic.")

peak_days = []
for y in sorted(march.index.year.unique()):
    g = march[march.index.year == y]
    if len(g) == 0:
        continue
    peak_ts = g.idxmax()
    peak_days.append(pd.Timestamp(peak_ts.date(), tz = "America/Los_Angeles"))

dom = np.array([x.day for x in peak_days])
med_dom = int(np.median(dom))
d_star = pd.Timestamp(f"2026-03-{med_dom:02d}", tz = "America/Los_Angeles")

print("d* (forecast date of March 2026 peak):", d_star.date().isoformat())
print("Historical March peak day-of-month values used:", sorted(dom.tolist()))

d* (forecast date of March 2026 peak): 2026-03-14
Historical March peak day-of-month values used: [14]


## Part c)

In [11]:
df = pd.read_csv(DATA_PATH)

d = pd.to_datetime(df["Date"], errors = "coerce")
he = pd.to_numeric(df["Hour"], errors = "coerce")
load = pd.to_numeric(df["CAISO Load (MW)"], errors = "coerce")
tmp = pd.DataFrame({"date": d, "he": he, "load": load}).dropna()

base = tmp["date"].dt.tz_localize("America/Los_Angeles")
ts = base + pd.to_timedelta(tmp["he"].astype(int), unit = "h")

hourly = pd.Series(tmp["load"].values, index = ts).sort_index()
hourly = hourly[~hourly.index.duplicated(keep = "last")].asfreq("H")
hourly = hourly.loc[hourly.index <= CUTOFF_END]

monthly_peaks = hourly.resample("MS").max().dropna()

print("Monthly peak months available:", len(monthly_peaks))
display(monthly_peaks.tail(12))

MIN_TRAIN = 6

rows = []
for i in range(MIN_TRAIN, len(monthly_peaks)):
    train = monthly_peaks.iloc[:i]
    test_ts = monthly_peaks.index[i]
    actual = float(monthly_peaks.iloc[i])

    model = ETSModel(train, error = "add", trend = "add", seasonal = None)
    fit = model.fit(disp = False)
    fc = float(fit.forecast(steps = 1).iloc[0])

    rows.append({"month": test_ts.date().isoformat(), "actual_mw": actual, "forecast_mw": fc})

bt = pd.DataFrame(rows)
if bt.empty:
    raise RuntimeError("Backtest produced no rows (need at least 7 months total).")

bt["abs_err"] = (bt["forecast_mw"] - bt["actual_mw"]).abs()
bt["ape_pct"] = bt["abs_err"] / bt["actual_mw"] * 100.0

mae = float(bt["abs_err"].mean())
mape = float(bt["ape_pct"].mean())

display(bt.tail(24))
print(f"Backtest (walk-forward monthly) MAE (MW): {mae:,.0f}")
print(f"Backtest (walk-forward monthly) MAPE (%): {mape:,.2f}")

Monthly peak months available: 13


2024-12-01 00:00:00-08:00    30265.19
2025-01-01 00:00:00-08:00    29173.90
2025-02-01 00:00:00-08:00    29126.53
2025-03-01 00:00:00-08:00    28127.03
2025-04-01 00:00:00-07:00    28444.29
2025-05-01 00:00:00-07:00    36265.24
2025-06-01 00:00:00-07:00    36284.07
2025-07-01 00:00:00-07:00    39655.38
2025-08-01 00:00:00-07:00    43922.82
2025-09-01 00:00:00-07:00    42416.44
2025-10-01 00:00:00-07:00    31473.68
2025-11-01 00:00:00-07:00    22638.12
Freq: MS, dtype: float64

,month,actual_mw,forecast_mw,abs_err,ape_pct
0,2025-05-01,36265.24,27745.362500,8519.877500,23.493233
1,2025-06-01,36284.07,32681.840422,3602.229578,9.927854
2,2025-07-01,39655.38,34077.926431,5577.453569,14.064809
3,2025-08-01,43922.82,40553.879485,3368.940515,7.670137
4,2025-09-01,42416.44,41716.997738,699.442262,1.648989
5,2025-10-01,31473.68,43488.619776,12014.939776,38.174563
6,2025-11-01,22638.12,42229.012885,19590.892885,86.539399


Backtest (walk-forward monthly) MAE (MW): 7,625
Backtest (walk-forward monthly) MAPE (%): 25.93


## Part d)

In [9]:
df = pd.read_csv(DATA_PATH)

d = pd.to_datetime(df["Date"], errors = "coerce")
he = pd.to_numeric(df["Hour"], errors = "coerce")
load = pd.to_numeric(df["CAISO Load (MW)"], errors = "coerce")
tmp = pd.DataFrame({"date": d, "he": he, "load": load}).dropna()

base = tmp["date"].dt.tz_localize("America/Los_Angeles")
ts = base + pd.to_timedelta(tmp["he"].astype(int), unit = "h")

hourly = pd.Series(tmp["load"].values, index = ts).sort_index()
hourly = hourly[~hourly.index.duplicated(keep = "last")].asfreq("H")
hourly = hourly.loc[hourly.index <= CUTOFF_END]

monthly_peaks = hourly.resample("MS").max().dropna()

model = ETSModel(monthly_peaks, error = "add", trend = "add", seasonal = None)
fit = model.fit(disp = False)

print("Final model: ETS with additive error + additive trend (Holt), NO seasonality")
print(fit.summary())

Final model: ETS with additive error + additive trend (Holt), NO seasonality
                                 ETS Results                                  
Dep. Variable:                      y   No. Observations:                   13
Model:                       ETS(AAN)   Log Likelihood                -127.957
Date:                Thu, 19 Feb 2026   AIC                            265.913
Time:                        04:29:06   BIC                            268.738
Sample:                    11-01-2024   HQIC                           265.332
                         - 11-01-2025   Scale                     20743425.363
Covariance Type:               approx                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
smoothing_level     0.9999      1.047      0.955      0.340      -1.052       3.052
smoothing_trend     0.7688      0.879  

## Part e)

In [10]:
df = pd.read_csv(DATA_PATH)

d = pd.to_datetime(df["Date"], errors = "coerce")
he = pd.to_numeric(df["Hour"], errors = "coerce")
load = pd.to_numeric(df["CAISO Load (MW)"], errors = "coerce")
tmp = pd.DataFrame({"date": d, "he": he, "load": load}).dropna()

base = tmp["date"].dt.tz_localize("America/Los_Angeles")
ts = base + pd.to_timedelta(tmp["he"].astype(int), unit = "h")

hourly = pd.Series(tmp["load"].values, index = ts).sort_index()
hourly = hourly[~hourly.index.duplicated(keep = "last")].asfreq("H")
hourly = hourly.loc[hourly.index <= CUTOFF_END]

monthly_peaks = hourly.resample("MS").max().dropna()

MIN_TRAIN = 6
rows = []
for i in range(MIN_TRAIN, len(monthly_peaks)):
    train = monthly_peaks.iloc[:i]
    actual = float(monthly_peaks.iloc[i])

    model_bt = ETSModel(train, error = "add", trend = "add", seasonal = None)
    fit_bt = model_bt.fit(disp = False)
    fc = float(fit_bt.forecast(steps = 1).iloc[0])
    rows.append((monthly_peaks.index[i], actual, fc))

bt = pd.DataFrame(rows, columns = ["month", "actual_mw", "forecast_mw"])
bt["abs_err"] = (bt["forecast_mw"] - bt["actual_mw"]).abs()
bt["ape_pct"] = bt["abs_err"] / bt["actual_mw"] * 100.0
mae = float(bt["abs_err"].mean())
mape = float(bt["ape_pct"].mean())

target_ms = pd.Timestamp("2026-03-01", tz = "America/Los_Angeles")
last_ms = monthly_peaks.index.max()
steps = int(target_ms.to_period("M").ordinal - last_ms.to_period("M").ordinal)
if steps <= 0:
    raise ValueError("Your data includes March 2026 or later (violates cutoff).")

model = ETSModel(monthly_peaks, error = "add", trend = "add", seasonal = None)
fit = model.fit(disp = False)
L_star = float(fit.forecast(steps = steps).iloc[-1])

march = hourly.dropna()
march = march[march.index.month == 3]
if march.empty:
    raise RuntimeError("No March history in data; cannot estimate d* with this method.")
peak_days = []
for y in sorted(march.index.year.unique()):
    peak_ts = march[march.index.year == y].idxmax()
    peak_days.append(pd.Timestamp(peak_ts.date(), tz = "America/Los_Angeles"))
dom = np.array([d.day for d in peak_days])
d_star = pd.Timestamp(f"2026-03-{int(np.median(dom)):02d}", tz = "America/Los_Angeles")

print(f"L* (MW) = {L_star:,.0f}")
print(f"d*      = {d_star.date().isoformat()}")
print(f"Backtest MAE (MW) = {mae:,.0f}")
print(f"Backtest MAPE (%) = {mape:,.2f}")
display(bt.tail(24))

L* (MW) = -12,369
d*      = 2026-03-14
Backtest MAE (MW) = 7,625
Backtest MAPE (%) = 25.93


C:\Users\ayerh\AppData\Local\Temp\ipykernel_22888\2593636610.py:38: UserWarning: Converting to Period representation will drop timezone information.
  steps = int(target_ms.to_period("M").ordinal - last_ms.to_period("M").ordinal)


,month,actual_mw,forecast_mw,abs_err,ape_pct
0,2025-05-01 00:00:00-07:00,36265.24,27745.362500,8519.877500,23.493233
1,2025-06-01 00:00:00-07:00,36284.07,32681.840422,3602.229578,9.927854
2,2025-07-01 00:00:00-07:00,39655.38,34077.926431,5577.453569,14.064809
3,2025-08-01 00:00:00-07:00,43922.82,40553.879485,3368.940515,7.670137
4,2025-09-01 00:00:00-07:00,42416.44,41716.997738,699.442262,1.648989
5,2025-10-01 00:00:00-07:00,31473.68,43488.619776,12014.939776,38.174563
6,2025-11-01 00:00:00-07:00,22638.12,42229.012885,19590.892885,86.539399
